<div class="alert alert-danger" role="alert">

**IN THIS NOTEBOOK WE ANALYZE AND PLOT TRANSFERS OF SIZE 20-50 MILLION EUROS FOR DIFFERENT YEARS**
    
</div>

**PREAMBLE**

In [28]:
import pandas as pd
#import fuzzy_pandas as fpd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px
import plotly

#for offline plotting using plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

**CLEANING STADIUM COORDINATES**

In [29]:
stadia = pd.read_csv("Data/Stadium_Coordinates_EDA.csv")

stadia['coordinates'] = stadia['coordinates'].str.replace("Point\(","") 
stadia['coordinates'] = stadia['coordinates'].str.replace("\)","") 

temp = stadia['coordinates'].str.split(expand = True)

temp = temp.rename(columns={0:'Longitude',1:'Latitude'})

a = temp[['Latitude','Longitude']].astype(float)

stadia = stadia.join(a)
stadia_cleaned = stadia.drop_duplicates(subset=['venueLabel'], keep="first", inplace=False)

stadia_cleaned = stadia_cleaned[['clubLabel','venueLabel','Latitude','Longitude']]

stadia_cleaned

,clubLabel,venueLabel,Latitude,Longitude
0,S.S.C. Napoli,Stadio San Paolo,40.827967,14.193008
2,Manchester United F.C.,Old Trafford,53.463056,-2.291389
3,SD Compostela,Estadio Multiusos de San Lázaro,42.882767,-8.516014
6,Sacramento Knights,Sleep Train Arena,38.649167,-121.518056
7,Novara Calcio,Stadio Silvio Piola (Novara),45.435833,8.595833
...,...,...,...,...
9035,FK Smiltene/BJSS,Teperis stadium,57.430749,25.920292
9036,Pyrgos Limassol Women FC,Pyrgos Community Stadium,34.746644,33.188163
9037,Juventude Sport Clube,Sanches de Miranda Stadium,38.561944,-7.913889
9038,Louisville City FC,Lynn Family Stadium,38.259590,-85.733264


**TRANSFER MARKT DATASET**

In [30]:
#Test Dataset
Test_1 = pd.read_csv("Data/transfer_data_Final_Master_2010_2011_Jul.csv")
Test_2 = pd.read_csv("Data/transfer_data_Final_Master_2011_Aug_2012_Jun.csv")
Test_3 = pd.read_csv("Data/transfer_data_Final_Master_2012_Jul_2015.csv")
Test_4 = pd.read_csv("Data/transfer_data_Final_Master_2016_Aug_Dec.csv")
Test_5 = pd.read_csv("Data/transfer_data_Final_Master_2016_Jan_Jul.csv")
Test_6 = pd.read_csv("Data/transfer_data_Final_Master_2017.csv")
Test_7 = pd.read_csv("Data/transfer_data_Final_Master_2018.csv")
Test_8 = pd.read_csv("Data/transfer_data_Final_Master_2019.csv")
Test_9 = pd.read_csv("Data/transfer_data_Final_Master_2005.csv")
Test_10 = pd.read_csv("Data/transfer_data_Final_Master_2006.csv")
Test_11 = pd.read_csv("Data/transfer_data_Final_Master_2008.csv")
Test_12 = pd.read_csv("Data/transfer_data_Final_Master_2009.csv")
Test_13 = pd.read_csv("Data/transfer_data_Final_Master_2007.csv")
Test_14 = pd.read_csv("Data/transfer_data_Final_Master_2007_Apr.csv")
Test_EDA = pd.concat([Test_1, Test_2, Test_3,
                      Test_4, Test_5, Test_6, 
                      Test_7, Test_8, Test_9, 
                      Test_10, Test_11, Test_12, 
                      Test_13, Test_14], ignore_index=True)

# Cleaning the name for the column Club left
New = Test_EDA['Club Left'].str.replace('\n', '')
New = New.to_frame(name = 'Club_Left')

# Cleaning the name for the column Club Joined
New1 = Test_EDA['Club Joined'].str.replace('\n', '')
New1 = New1.to_frame(name = 'Club_Joined')

# Cleanning Transfer Data
New2 = Test_EDA['Transfer'].str.replace('?', '0')
New2 = New2.to_frame(name = 'Transfer_Fee')

New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Free transfer","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("-","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(",","")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace("Draft","0")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" mil. €","0000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" K €","000")
New2["Transfer_Fee"]=New2["Transfer_Fee"].str.replace(" €","")

# Cleaning Market Valuation data
New3 = Test_EDA['Market Valuation'].str.replace(',', '')
New3 = New3.to_frame(name = 'Market_Valuation')
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" mil. €","0000")
New3["Market_Valuation"]=New3["Market_Valuation"].str.replace(" K €","000")

#Replacing with original data
Test_EDA["Club Left"]=New["Club_Left"]
Test_EDA["Club Joined"]=New1["Club_Joined"]
Test_EDA["Transfer"]=New2["Transfer_Fee"]
Test_EDA["Market Valuation"]=New3["Market_Valuation"]

# Converting the Date from object ot a datetime format
Test_EDA['Date'] = pd.to_datetime(Test_EDA['Date'], errors='coerce')

# Removing retired players
Test_EDA.rename(columns={"Club Joined": "Club_Joined"}, inplace=True)
Test_EDA=Test_EDA[~Test_EDA.Club_Joined.str.contains("Retired")]
Test_EDA.rename(columns={"Club_Joined": "Club Joined"}, inplace=True)

Test_EDA.reset_index(inplace=True)

#Converting to float
Test_EDA["Transfer"]=Test_EDA['Transfer'].apply(lambda x:float(x))
Test_EDA["Market Valuation"]=Test_EDA["Market Valuation"].apply(lambda x:float(x))
Test_EDA["Age"]=Test_EDA['Age'].apply(lambda x:float(x))

#Adding surplus/deficit column
Test_EDA.rename(columns={"Market Valuation": "Market_Valuation"}, inplace=True)
Test_EDA["Surplus/Deficit"]=Test_EDA.Transfer-Test_EDA.Market_Valuation
Test_EDA.rename(columns={"Market_Valuation": "Market Valuation"}, inplace=True)

#Adding year column
Test_EDA["Year"]=pd.DatetimeIndex(Test_EDA["Date"]).year

Test_EDA

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,Market Valuation,Transfer,Surplus/Deficit,Year
0,0,2010-01-01,Douglas Costa,19.0,Right Winger,Brazil,Grêmio,Série A,Shakhtar D.,NaN,4000000.0,8000000.0,4000000.0,2010
1,1,2010-01-01,Florent Sinama-Pongolle,25.0,Second Striker,France,Atlético Madrid,LaLiga,Sporting CP,NaN,7000000.0,6500000.0,-500000.0,2010
2,2,2010-01-01,Alex Teixeira,19.0,Left Winger,Brazil,Vasco da Gama,Série A,Shakhtar D.,NaN,3500000.0,6000000.0,2500000.0,2010
3,3,2010-01-01,Keisuke Honda,23.0,Attacking Midfield,Japan,VVV-Venlo,Eredivisie,CSKA Moscow,NaN,3000000.0,6000000.0,3000000.0,2010
4,4,2010-01-01,Younès Kaboul,23.0,Centre-Back,France,Portsmouth,Premier League,Spurs,NaN,5700000.0,5900000.0,200000.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61835,63255,2007-09-25,Thomas Sowunmi,29.0,Centre-Forward,Hungary,Without Club,NaN,Vasas FC,NaN,300000.0,0.0,-300000.0,2007
61836,63256,2007-09-28,Onur Karali,25.0,Midfielder,Turkey,Osmaniyespor,Turkey\t\t,Altinova Bld,NaN,200000.0,0.0,-200000.0,2007
61837,63257,2007-09-28,Enver Isik,22.0,Right Winger,Turkey,Kayserispor,Süper Lig,Göztepe,NaN,250000.0,0.0,-250000.0,2007
61838,63258,2007-09-28,Mustafa Özzengi,20.0,Goalkeeper,Turkey,Galatasaray U21,Turkey\t\t,Bakirköyspor,NaN,50000.0,0.0,-50000.0,2007


**EXTRACTING LIST OF CLUBS IN THE TRANSFER MARKT DATASET**

In [31]:
Test_EDA[['Club Left','Club Joined']]

clubs_left_names = Test_EDA['Club Left'].unique()
clubs_joined_names =Test_EDA['Club Joined'].unique()

clubs_left_names = clubs_left_names.tolist()
clubs_joined_names = clubs_joined_names.tolist()

transfer_markt_club_names = np.unique(clubs_left_names + clubs_joined_names)

transfer_markt_club_names = transfer_markt_club_names.tolist()

df_club_names = pd.Series(transfer_markt_club_names)
df_club_names = df_club_names.to_frame()
df_club_names = df_club_names.rename(columns={0:'Club Name'})
df_club_names

,Club Name
0,FK Uzhgorod
1,Gharb Rangers
2,Goyang Zaicro
3,Montello
4,PJ Rangers
...,...
11335,Ümraniyespor
11336,Ünye 1957
11337,Ünyespor
11338,Łagiewniki


**FUZZY MATCHING**

In [32]:
merged_df = pd.read_csv("Data/club_stadium_coordinates_Final.csv")

**CREATING THE MERGED MASTER DATASETS**

In [33]:
final_DataFrame_1 = pd.merge(Test_EDA,merged_df,
                    how='inner',
                    left_on = 'Club Left',
                    right_on='Club Name TM')

final_DataFrame_2 = pd.merge(final_DataFrame_1, merged_df,
                    how='inner',
                    left_on = 'Club Joined',
                    right_on='Club Name TM',
                    suffixes = ('_Left','_Joined'))

final_DataFrame_2 = final_DataFrame_2.rename(columns={'Club Name TM_Joined':'Club_Name_TM_Joined', 
                                                      'Club Name TM_Left':'Club_Name_TM_Left',
                                                      'Club Name Wikidata_Joined':'Club_Name_Wikidata_Joined',
                                                      'Club Name Wikidata_Left':'Club_Name_Wikidata_Left'})

#final_DataFrame_2 = final_DataFrame_2[final_DataFrame_2.Nationality=='China']

final_DataFrame_2

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_TM_Left,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined
0,766,2010-03-10,Aleksandr Marenich,20.0,Left Winger,Russia,FK Moskau,Russia\t\t,Spartak V.,NaN,...,FK Moskau,FK Khazar Lankaran,Lankaran City Stadium,38.747189,48.857131,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
1,10833,2013-01-23,Dmitri Tikhiy,20.0,Right-Back,Russia,Luch,PFL Ost,Spartak V.,NaN,...,Luch,Deportivo Toluca F.C.,Nemesio Díez Stadium,19.287286,-99.666794,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
2,518,2010-02-01,Ivan Stoyanov,26.0,Right Winger,Bulgaria,CSKA Sofia,Friendlies,Spartak V.,NaN,...,CSKA Sofia,PFC CSKA Sofia,Balgarska Armia Stadium,42.684258,23.339769,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
3,698,2010-03-01,Abdoul Gafar Mamah,24.0,Right-Back,Togo,Sheriff,Divizia Nationala,Spartak V.,NaN,...,Sheriff,FC Sheriff Tiraspol,Sheriff Stadium,46.838333,29.557500,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
4,3009,2011-01-19,Dmitri Grachev,27.0,Centre-Back,Russia,Saturn,Russia\t\t,Spartak V.,NaN,...,Saturn,FC Saturn Ramenskoye,Saturn Stadium,55.577222,38.226944,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,38178,2017-08-01,Andrei Sintean,18.0,Centre-Forward,Romania,Poli Timisoara,Liga 1,Slavia Prague B,NaN,...,Poli Timisoara,FC Politehnica Timișoara,Stadionul Dan Păltinișanu,45.740556,21.244167,Slavia Prague B,SK Slavia Prague,Sinobo Stadium,50.066944,14.471389
2506,49896,2019-03-03,Jamshid Khasanov,30.0,Attacking Midfield,Uzbekistan,AGMG Olmaliq,Superliga,Kokand 1912,NaN,...,AGMG Olmaliq,Nîmes Olympique,Stade des Costières,43.815972,4.359278,Kokand 1912,Békéscsaba 1912 Előre SE,Stadion Kórház utcai,46.686111,21.105000
2507,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,SC Longfor,Orlando City SC,Camping World Stadium,28.539167,-81.402778,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661
2508,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,SC Longfor,Orlando City SC,ESPN Wide World of Sports Complex,28.337083,-81.556000,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661


**CREATING THE DATA SET FOR THE NODES**

In [34]:
merged_df

Transfer_Value = final_DataFrame_2.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus = final_DataFrame_2.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_x = pd.merge(Transfer_Surplus,Transfer_Value,
                    how='outer',
                    on = 'Club Joined')

merged_df_2 = pd.merge(merged_df,merged_df_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2 = merged_df_2.fillna(0)

merged_df_2['Transfer'] = merged_df_2['Transfer']

merged_df_2 

,Club Name TM,Club Name Wikidata,venueLabel,Latitude,Longitude,Club Joined,Surplus/Deficit,Transfer
0,1.FC Heidenheim,1. FC Heidenheim,Voith-Arena,48.668500,10.139300,1.FC Heidenheim,-775000.0,2400000.0
1,1.FC K'lautern,Cusco FC,Estadio Garcilaso,-13.525000,-71.966000,1.FC K'lautern,-2350000.0,4550000.0
2,FC Pune City,Cusco FC,Estadio Garcilaso,-13.525000,-71.966000,FC Pune City,-3600000.0,0.0
3,Perugia,A.C. Perugia Calcio,Stadio Renato Curi,43.106111,12.357222,Perugia,-2200000.0,2200000.0
4,AC Ajaccio,A.C. Ajaccio,Stade François Coty,41.930761,8.776444,AC Ajaccio,-6150000.0,200000.0
...,...,...,...,...,...,...,...,...
371,YB Funde,Sudet,Kouvolan keskuskenttä,60.871944,26.713333,YB Funde,-1500000.0,0.0
372,YC Helanshan,Cheonan FC,Cheongju Sports Complex,36.637851,127.472375,YC Helanshan,-125000.0,0.0
373,Zawisza,Zawisza Bydgoszcz,Zdzisław Krzyszkowiak Stadium,53.145556,18.020556,Zawisza,-1200000.0,0.0
374,Zulte Waregem,S.V. Zulte Waregem,Regenboogstadion,50.883056,3.428889,Zulte Waregem,-2075000.0,1500000.0


**FINAL DATAFRAME FOR THE EDGES**

In [35]:
final_DataFrame_2 = pd.merge(final_DataFrame_2,Transfer_Value,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_y
0,766,2010-03-10,Aleksandr Marenich,20.0,Left Winger,Russia,FK Moskau,Russia\t\t,Spartak V.,NaN,...,FK Khazar Lankaran,Lankaran City Stadium,38.747189,48.857131,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
1,10833,2013-01-23,Dmitri Tikhiy,20.0,Right-Back,Russia,Luch,PFL Ost,Spartak V.,NaN,...,Deportivo Toluca F.C.,Nemesio Díez Stadium,19.287286,-99.666794,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
2,518,2010-02-01,Ivan Stoyanov,26.0,Right Winger,Bulgaria,CSKA Sofia,Friendlies,Spartak V.,NaN,...,PFC CSKA Sofia,Balgarska Armia Stadium,42.684258,23.339769,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
3,698,2010-03-01,Abdoul Gafar Mamah,24.0,Right-Back,Togo,Sheriff,Divizia Nationala,Spartak V.,NaN,...,FC Sheriff Tiraspol,Sheriff Stadium,46.838333,29.557500,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
4,3009,2011-01-19,Dmitri Grachev,27.0,Centre-Back,Russia,Saturn,Russia\t\t,Spartak V.,NaN,...,FC Saturn Ramenskoye,Saturn Stadium,55.577222,38.226944,Spartak V.,K.V.C. Westerlo,Het Kuipje,51.094722,4.928333,2350000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2505,38178,2017-08-01,Andrei Sintean,18.0,Centre-Forward,Romania,Poli Timisoara,Liga 1,Slavia Prague B,NaN,...,FC Politehnica Timișoara,Stadionul Dan Păltinișanu,45.740556,21.244167,Slavia Prague B,SK Slavia Prague,Sinobo Stadium,50.066944,14.471389,25000.0
2506,49896,2019-03-03,Jamshid Khasanov,30.0,Attacking Midfield,Uzbekistan,AGMG Olmaliq,Superliga,Kokand 1912,NaN,...,Nîmes Olympique,Stade des Costières,43.815972,4.359278,Kokand 1912,Békéscsaba 1912 Előre SE,Stadion Kórház utcai,46.686111,21.105000,0.0
2507,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,Orlando City SC,Camping World Stadium,28.539167,-81.402778,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661,0.0
2508,30578,2016-12-31,Liangxuan Gong,23.0,Defender,China,SC Longfor,League Two South,Evergrande Res.,NaN,...,Orlando City SC,ESPN Wide World of Sports Complex,28.337083,-81.556000,Evergrande Res.,Guangzhou Evergrande Taobao Football Club,Tianhe Stadium,23.138031,113.324661,0.0


**GRAPH RENDERING**

**IN THE ANALYSIS THAT FOLLOWS WE RESTRICT OUR FOCUS MEDIUM-LARGE TRANSFERS WHICH RANGE BETWEEN 20 MILLION EUROS AND 50 MILLION EUROS**

In [36]:
final_DataFrame_2=final_DataFrame_2[(final_DataFrame_2['Transfer_y']>=20000000) & (final_DataFrame_2['Transfer_y']<=50000000)]
final_DataFrame_2.rename(columns={"Transfer_y": "Transfer"}, inplace=True)
final_DataFrame_2

/home/anirudhpisharam/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4223: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer
86,18744,2014-07-22,Sokratis Dioudis,21.0,Goalkeeper,Greece,Aris Saloniki,Greece\t\t,Club Brugge,NaN,...,Aris Thessaloniki F.C.,Kleanthis Vikelidis Stadium,40.600069,22.969500,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
87,61559,2009-08-30,Dorge Kouemaha,26.0,Centre-Forward,Cameroon,MSV Duisburg,2. Bundesliga,Club Brugge,NaN,...,MSV Duisburg,Schauinsland-Reisen-Arena,51.409028,6.778639,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
88,52088,2019-07-10,Simon Deli,27.0,Centre-Back,Cote d'Ivoire,Slavia Prague,Fortuna Liga,Club Brugge,NaN,...,SK Slavia Prague,Sinobo Stadium,50.066944,14.471389,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
89,35164,2017-01-31,Dorin Rotariu,21.0,Right Winger,Romania,Dinamo Bukarest,Liga 1 - Championship group,Club Brugge,NaN,...,FC Dinamo-Polyot Bishkek,Dynamo Stadion,42.837222,74.600833,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
90,34169,2016-07-23,Anthony Limbombe,22.0,Left Winger,Belgium,NEC Nijmegen,Eredivisie,Club Brugge,NaN,...,Nui,Tuvalu Sports Ground,-8.523889,179.198611,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2252,16397,2014-01-30,David Pollet,25.0,Centre-Forward,Belgium,RSC Charleroi,Jupiler Pro League,RSC Anderlecht,NaN,...,R.O.C. de Charleroi-Marchienne,Stade de la Neuville,50.408524,4.456877,RSC Anderlecht,R.S.C. Anderlecht,Constant Vanden Stock Stadium,50.834167,4.298333,25050000.0
2253,25183,2015-08-07,Kara,25.0,Centre-Back,Senegal,KRC Genk,Jupiler Pro League,RSC Anderlecht,NaN,...,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,RSC Anderlecht,R.S.C. Anderlecht,Constant Vanden Stock Stadium,50.834167,4.298333,25050000.0
2254,59727,2009-01-01,Tom De Sutter,23.0,Centre-Forward,Belgium,Cercle Brugge,Jupiler Pro League,RSC Anderlecht,NaN,...,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,RSC Anderlecht,R.S.C. Anderlecht,Constant Vanden Stock Stadium,50.834167,4.298333,25050000.0
2255,22308,2015-02-01,Idrissa Sylla,24.0,Centre-Forward,Guinea,Zulte Waregem,Jupiler Pro League,RSC Anderlecht,NaN,...,S.V. Zulte Waregem,Regenboogstadion,50.883056,3.428889,RSC Anderlecht,R.S.C. Anderlecht,Constant Vanden Stock Stadium,50.834167,4.298333,25050000.0


**FIRST WE LOOK AT YEARS BETWEEN 2005 AND 2010**

In [37]:
years = [2005, 2010]

final_DataFrame_2005_2010 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]
final_DataFrame_2005_2010

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,Club_Name_Wikidata_Left,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer
91,1192,2010-07-12,Wilfried Dalmat,27.0,Right Winger,Saint-Martin,Standard Liège,Jupiler Pro League,Club Brugge,NaN,...,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0
932,269,2010-01-19,Luigi Pieroni,29.0,Centre-Forward,Belgium,Valenciennes FC,Ligue 1,KAA Gent,NaN,...,Valenciennes F.C.,Stade du Hainaut,50.348486,3.529336,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0
940,56353,2005-08-15,Christophe Grégoire,25.0,Left Winger,Belgium,RSC Anderlecht,Jupiler Pro League,KAA Gent,NaN,...,R.S.C. Anderlecht,Constant Vanden Stock Stadium,50.834167,4.298333,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0
1441,1542,2010-07-30,Luigi Pieroni,29.0,Centre-Forward,Belgium,KAA Gent,Jupiler Pro League,Standard Liège,NaN,...,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0
2157,56378,2005-08-20,Bob Peeters,31.0,Centre-Forward,Belgium,Millwall,Championship,KRC Genk,NaN,...,Millwall F.C.,The Den,51.485833,-0.050833,KRC Genk,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,41550000.0
2159,56273,2005-08-01,Gonzague Vandooren,25.0,Left-Back,Belgium,Standard Liège,Jupiler Pro League,KRC Genk,NaN,...,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,KRC Genk,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,41550000.0


**CREATING THE DATASET FOR THE NODES FOR YEARS 2005-2010**

In [38]:
merged_df

Transfer_Value_2005_2010 = final_DataFrame_2005_2010.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2005_2010 = final_DataFrame_2005_2010.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2005_2010_x = pd.merge(Transfer_Surplus_2005_2010,Transfer_Value_2005_2010,
                    how='outer',
                    on = 'Club Joined')

merged_df_2005_2010 = pd.merge(merged_df,merged_df_2005_2010_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2005_2010 = merged_df_2005_2010.fillna(0)

merged_df_2005_2010['Transfer'] = merged_df_2005_2010['Transfer']

merged_df_2005_2010

,Club Name TM,Club Name Wikidata,venueLabel,Latitude,Longitude,Club Joined,Surplus/Deficit,Transfer
0,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,Club Brugge,-2500000.0,35215000.0
1,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,KAA Gent,-1480000.0,47860000.0
2,KRC Genk,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,KRC Genk,-1300000.0,83100000.0
3,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,Standard Liège,-650000.0,29850000.0


**MASTER DATASET FOR YEARS 2005-2010**

In [39]:
final_DataFrame_2005_2010 = pd.merge(final_DataFrame_2005_2010,Transfer_Value_2005_2010,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2005_2010

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Transfer_y
0,1192,2010-07-12,Wilfried Dalmat,27.0,Right Winger,Saint-Martin,Standard Liège,Jupiler Pro League,Club Brugge,NaN,...,Stade Maurice Dufrasne,50.609908,5.543344,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,35215000.0
1,269,2010-01-19,Luigi Pieroni,29.0,Centre-Forward,Belgium,Valenciennes FC,Ligue 1,KAA Gent,NaN,...,Stade du Hainaut,50.348486,3.529336,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,47860000.0
2,56353,2005-08-15,Christophe Grégoire,25.0,Left Winger,Belgium,RSC Anderlecht,Jupiler Pro League,KAA Gent,NaN,...,Constant Vanden Stock Stadium,50.834167,4.298333,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,47860000.0
3,1542,2010-07-30,Luigi Pieroni,29.0,Centre-Forward,Belgium,KAA Gent,Jupiler Pro League,Standard Liège,NaN,...,Jules Ottenstadion,51.031389,3.766111,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,29850000.0
4,56378,2005-08-20,Bob Peeters,31.0,Centre-Forward,Belgium,Millwall,Championship,KRC Genk,NaN,...,The Den,51.485833,-0.050833,KRC Genk,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,41550000.0,83100000.0
5,56273,2005-08-01,Gonzague Vandooren,25.0,Left-Back,Belgium,Standard Liège,Jupiler Pro League,KRC Genk,NaN,...,Stade Maurice Dufrasne,50.609908,5.543344,KRC Genk,R.C. Lens,Stade Félix-Bollaert,50.432778,2.814722,41550000.0,83100000.0


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 20-50 million euros in the years 2005-2010 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    We see that number of transfers in the range 20-50 million euros were not that high in number and were mostly restricted to the European clubs asserting that the European leagues have always been an integral part of the global football presence. 

    
</div>

In [40]:
club_left_lat = final_DataFrame_2005_2010.Latitude_Left
club_left_lon = final_DataFrame_2005_2010.Longitude_Left

club_join_lat = final_DataFrame_2005_2010.Latitude_Joined
club_join_lon = final_DataFrame_2005_2010.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2005_2010.Longitude,
            lat = merged_df_2005_2010.Latitude,
            text = merged_df_2005_2010['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2005_2010['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2005_2010['Surplus/Deficit'].min(),
            color = merged_df_2005_2010['Surplus/Deficit'],
            cmax = merged_df_2005_2010['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2005_2010)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2005_2010['Longitude_Left'][i], final_DataFrame_2005_2010['Longitude_Joined'][i]],
            lat = [final_DataFrame_2005_2010['Latitude_Left'][i], final_DataFrame_2005_2010['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2005_2010['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 20M to 50M (2005-2010)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_20M_50M_2005_2010.html')

'Transfer_Size_20M_50M_2005_2010.html'

**NOW WE LOOK AT YEARS 2011-2015**

In [41]:
years = [2011, 2015]

final_DataFrame_2011_2015 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2011_2015 = final_DataFrame_2011_2015.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2011_2015 = final_DataFrame_2011_2015.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2011_2015_x = pd.merge(Transfer_Surplus_2011_2015,Transfer_Value_2011_2015,
                    how='outer',
                    on = 'Club Joined')

merged_df_2011_2015 = pd.merge(merged_df,merged_df_2011_2015_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2011_2015 = merged_df_2011_2015.fillna(0)

merged_df_2011_2015['Transfer'] = merged_df_2011_2015['Transfer']

final_DataFrame_2011_2015 = pd.merge(final_DataFrame_2011_2015,Transfer_Value_2011_2015,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2011_2015

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Transfer_y
0,5593,2011-11-01,Bojan Jorgacevic,29.0,Goalkeeper,Serbia,KAA Gent,Jupiler Pro League,Club Brugge,NaN,...,Jules Ottenstadion,51.031389,3.766111,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,70430000.0
1,25347,2015-08-13,Michaël Cordier,31.0,Goalkeeper,Belgium,KVC Westerlo,Jupiler Pro League,Club Brugge,NaN,...,Het Kuipje,51.094722,4.928333,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,70430000.0
2,5134,2011-08-30,Rafinha,29.0,Right-Back,Brazil,HJK Helsinki,Veikkausliiga,KAA Gent,NaN,...,Bolt Arena,60.187500,24.922500,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,47860000.0
3,24439,2015-07-08,Hatem Abd Elhamed,24.0,Centre-Back,Israel,FC Ashdod,Liga Leumit,KAA Gent,NaN,...,Allianz Arena,48.218775,11.624753,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,47860000.0
4,2675,2011-01-03,Razak Boukari,23.0,Right Winger,Togo,Lens,Ligue 1,Stade Rennais,NaN,...,Stade Félix-Bollaert,50.432778,2.814722,Stade Rennais,Stade Rennais F.C.,Roazhon Park,48.107472,-1.712861,48200000.0,96400000.0
5,22333,2015-02-02,Dimitri Cavaré,19.0,Right-Back,Guadeloupe,Lens,Ligue 1,Stade Rennais,NaN,...,Stade Félix-Bollaert,50.432778,2.814722,Stade Rennais,Stade Rennais F.C.,Roazhon Park,48.107472,-1.712861,48200000.0,96400000.0
6,26720,2015-10-14,Réginal Goreux,27.0,Right-Back,Haiti,Without Club,NaN,Standard Liège,NaN,...,Estadio Tiburcio Carías Andino beach,14.098403,-87.203978,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,119400000.0
7,24310,2015-07-03,Darwin Andrade,24.0,Left-Back,Colombia,Újpest FC,NB I.,Standard Liège,NaN,...,Ferenc Szusza Stadium,47.574892,19.084650,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,119400000.0
8,4892,2011-08-19,Maor Buzaglo,23.0,Right Winger,Israel,M. Tel Aviv,Ligat ha'Al,Standard Liège,NaN,...,Bloomfield Stadium,32.051750,34.761625,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,119400000.0
9,22004,2015-01-20,Alexander Scholz,22.0,Centre-Back,Denmark,KSC Lokeren,Jupiler Pro League,Standard Liège,NaN,...,Daknamstadion,51.116389,3.987222,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,119400000.0


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 20-50 million euros in the years 2011-2015 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    We see a massive increase in the number of transfers in this five year period. American clubs started getting involved in the transfer market and another important observation is that the activity of Asian clubs (especially China based clubs) has increased massively.

    
</div>

In [42]:
club_left_lat = final_DataFrame_2011_2015.Latitude_Left
club_left_lon = final_DataFrame_2011_2015.Longitude_Left

club_join_lat = final_DataFrame_2011_2015.Latitude_Joined
club_join_lon = final_DataFrame_2011_2015.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2011_2015.Longitude,
            lat = merged_df_2011_2015.Latitude,
            text = merged_df_2011_2015['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2011_2015['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2011_2015['Surplus/Deficit'].min(),
            color = merged_df_2011_2015['Surplus/Deficit'],
            cmax = merged_df_2011_2015['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2011_2015)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2011_2015['Longitude_Left'][i], final_DataFrame_2011_2015['Longitude_Joined'][i]],
            lat = [final_DataFrame_2011_2015['Latitude_Left'][i], final_DataFrame_2011_2015['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2011_2015['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 20M to 50M (2011-2015)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_20M_50M_2011_2015.html')

'Transfer_Size_20M_50M_2011_2015.html'

**CONTINUING WE NOW LOOK AT 2016-2017**

In [43]:
years = [2016, 2017]

final_DataFrame_2016_2017 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2016_2017 = final_DataFrame_2016_2017.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2016_2017 = final_DataFrame_2016_2017.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2016_2017_x = pd.merge(Transfer_Surplus_2016_2017,Transfer_Value_2016_2017,
                    how='outer',
                    on = 'Club Joined')

merged_df_2016_2017 = pd.merge(merged_df,merged_df_2016_2017_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2016_2017 = merged_df_2016_2017.fillna(0)

merged_df_2016_2017['Transfer'] = merged_df_2016_2017['Transfer']

final_DataFrame_2016_2017 = pd.merge(final_DataFrame_2016_2017,Transfer_Value_2016_2017,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2016_2017

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Transfer_y
0,35164,2017-01-31,Dorin Rotariu,21.0,Right Winger,Romania,Dinamo Bukarest,Liga 1 - Championship group,Club Brugge,NaN,...,Dynamo Stadion,42.837222,74.600833,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,105645000.0
1,34169,2016-07-23,Anthony Limbombe,22.0,Left Winger,Belgium,NEC Nijmegen,Eredivisie,Club Brugge,NaN,...,Tuvalu Sports Ground,-8.523889,179.198611,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,105645000.0
2,31077,2016-01-20,Benoît Poulain,28.0,Centre-Back,France,KV Kortrijk,Jupiler Pro League,Club Brugge,NaN,...,Guldensporen Stadion,50.830278,3.248889,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,105645000.0
3,27877,2016-08-05,Ofir Davidzada,25.0,Left-Back,Israel,H. Beer Sheva,Ligat ha'Al,KAA Gent,NaN,...,Vasermil Stadium,31.253794,34.785964,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,95720000.0
4,31203,2016-01-25,Rob Schoofs,21.0,Central Midfield,Belgium,Sint-Truiden,Jupiler Pro League,KAA Gent,NaN,...,Stayen,50.813611,5.165278,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,95720000.0
5,34798,2017-01-17,Birger Verstraete,22.0,Defensive Midfield,Belgium,KV Kortrijk,Jupiler Pro League,KAA Gent,NaN,...,Guldensporen Stadion,50.830278,3.248889,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,95720000.0
6,35049,2017-01-27,Samuel Gigot,23.0,Centre-Back,France,KV Kortrijk,Jupiler Pro League,KAA Gent,NaN,...,Guldensporen Stadion,50.830278,3.248889,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,95720000.0
7,38839,2017-08-28,Tomas Koubek,25.0,Goalkeeper,Czech Republic,Sparta Praha,Fortuna Liga,Stade Rennais,NaN,...,Wisner Stadium,42.648300,-83.306800,Stade Rennais,Stade Rennais F.C.,Roazhon Park,48.107472,-1.712861,48200000.0,48200000.0
8,34273,2016-07-27,Isaac Mbenza,20.0,Right Winger,Belgium,Valenciennes FC,Ligue 2,Standard Liège,NaN,...,Stade du Hainaut,50.348486,3.529336,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,298500000.0
9,37447,2017-07-03,Paul-José Mpoku,25.0,Left Winger,DR Congo,Chievo Verona,Serie A,Standard Liège,NaN,...,Stadio Marc'Antonio Bentegodi,45.435356,10.968647,Standard Liège,Standard Liège,Stade Maurice Dufrasne,50.609908,5.543344,29850000.0,298500000.0


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 20-50 million euros in the years 2016-2017 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    This two year period presents an interesting picture. Number of transfers have not gone down from the previous five year period and an important point to note is that the surplus of the clubs have gone down pretty evenly across the world. This means that there has been a drop in the overspending and clubs have started spending more efficiently. This could be because the clubs and their managements have finally started analysing the tranfer market data and incorporating it in their trades.
    
</div>

In [44]:
club_left_lat = final_DataFrame_2016_2017.Latitude_Left
club_left_lon = final_DataFrame_2016_2017.Longitude_Left

club_join_lat = final_DataFrame_2016_2017.Latitude_Joined
club_join_lon = final_DataFrame_2016_2017.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2016_2017.Longitude,
            lat = merged_df_2016_2017.Latitude,
            text = merged_df_2016_2017['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2016_2017['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2016_2017['Surplus/Deficit'].min(),
            color = merged_df_2016_2017['Surplus/Deficit'],
            cmax = merged_df_2016_2017['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2016_2017)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2016_2017['Longitude_Left'][i], final_DataFrame_2016_2017['Longitude_Joined'][i]],
            lat = [final_DataFrame_2016_2017['Latitude_Left'][i], final_DataFrame_2016_2017['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2016_2017['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 20M to 50M (2016-2017)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_20M_50M_2016_2017.html')

'Transfer_Size_20M_50M_2016_2017.html'

**FINALLY WE LOOK AT YEARS 2018-2019**

In [45]:
years = [2018, 2019]

final_DataFrame_2018_2019 = final_DataFrame_2[final_DataFrame_2.Year.isin(years)]

merged_df

Transfer_Value_2018_2019 = final_DataFrame_2018_2019.groupby('Club Joined').agg({'Transfer':'sum'}).reset_index()

Transfer_Surplus_2018_2019 = final_DataFrame_2018_2019.groupby('Club Joined').agg({'Surplus/Deficit':'sum'})

merged_df_2018_2019_x = pd.merge(Transfer_Surplus_2018_2019,Transfer_Value_2018_2019,
                    how='outer',
                    on = 'Club Joined')

merged_df_2018_2019 = pd.merge(merged_df,merged_df_2018_2019_x,
                    how='inner',
                    left_on = 'Club Name TM',
                    right_on = 'Club Joined')

merged_df_2018_2019 = merged_df_2018_2019.fillna(0)

merged_df_2018_2019['Transfer'] = merged_df_2018_2019['Transfer']

final_DataFrame_2018_2019 = pd.merge(final_DataFrame_2018_2019,Transfer_Value_2018_2019,
                    how='inner',
                    on = 'Club Joined')
final_DataFrame_2018_2019

,index,Date,Name,Age,Position,Nationality,Club Left,League Left,Club Joined,League Joined,...,venueLabel_Left,Latitude_Left,Longitude_Left,Club_Name_TM_Joined,Club_Name_Wikidata_Joined,venueLabel_Joined,Latitude_Joined,Longitude_Joined,Transfer_x,Transfer_y
0,52088,2019-07-10,Simon Deli,27.0,Centre-Back,Cote d'Ivoire,Slavia Prague,Fortuna Liga,Club Brugge,NaN,...,Sinobo Stadium,50.066944,14.471389,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
1,42074,2018-01-29,Alexander Scholz,25.0,Centre-Back,Denmark,Standard Liège,Jupiler Pro League,Club Brugge,NaN,...,Stade Maurice Dufrasne,50.609908,5.543344,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
2,52963,2019-08-14,Federico Ricca,24.0,Left-Back,Uruguay,Málaga CF,LaLiga2,Club Brugge,NaN,...,La Rosaleda Stadium,36.734092,-4.426422,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
3,45887,2018-08-22,Kaveh Rezaei,26.0,Centre-Forward,Iran,RSC Charleroi,Jupiler Pro League,Club Brugge,NaN,...,Stade de la Neuville,50.408524,4.456877,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
4,41546,2018-01-08,Ivan Tomecak,28.0,Right-Back,Croatia,KV Mechelen,Jupiler Pro League,Club Brugge,NaN,...,AFAS Stadion KV Mechelen,51.037222,4.486389,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
5,52061,2019-07-09,David Okereke,21.0,Centre-Forward,Nigeria,Spezia Calcio,Serie B,Club Brugge,NaN,...,Stadio Alberto Picco,44.102222,9.808611,Club Brugge,Club Brugge K.V.,Jan Breydel Stadium,51.193269,3.181311,35215000.0,211290000.0
6,52312,2019-07-19,Michael Ngadeu Ngadjui,28.0,Centre-Back,Cameroon,Slavia Prague,Fortuna Liga,KAA Gent,NaN,...,Sinobo Stadium,50.066944,14.471389,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,119650000.0
7,45089,2018-07-21,Vadis Odjidja-Ofoe,29.0,Central Midfield,Belgium,Olympiacos,Super League 1,KAA Gent,NaN,...,Karaiskakis Stadium,37.946169,23.664536,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,119650000.0
8,45012,2018-07-18,Giorgi Kvilitaia,24.0,Centre-Forward,Georgia,Rapid Vienna,Bundesliga,KAA Gent,NaN,...,Stadionul Regie,44.448194,26.052872,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,119650000.0
9,48960,2019-01-25,Roman Bezus,28.0,Attacking Midfield,Ukraine,Sint-Truiden,Jupiler Pro League,KAA Gent,NaN,...,Stayen,50.813611,5.165278,KAA Gent,K.A.A. Gent,Jules Ottenstadion,51.031389,3.766111,23930000.0,119650000.0


<div class="alert alert-warning" role="alert">

**Method** <br>
In following graph, we give a pictorial representation of transfers of size 20-50 million euros in the years 2018-2019 <br><br>

</div>

<div class="alert alert-success" role="alert">
    
**Inference**<br>
    A more cluttered network of transfers implies that the numbers have increased since 2017. Also, clubs have again started spending big bucks as shown by high surplus figures.

    
</div>

In [46]:
club_left_lat = final_DataFrame_2018_2019.Latitude_Left
club_left_lon = final_DataFrame_2018_2019.Longitude_Left

club_join_lat = final_DataFrame_2018_2019.Latitude_Joined
club_join_lon = final_DataFrame_2018_2019.Longitude_Joined

fig = go.Figure()

fig.add_trace(
        go.Scattergeo(
            lon = merged_df_2018_2019.Longitude,
            lat = merged_df_2018_2019.Latitude,
            text = merged_df_2018_2019['Club Name Wikidata'],
            hoverinfo = 'text',
            mode = 'markers',
            marker = dict(
                size = np.log((merged_df_2018_2019['Transfer']+100)*100000)*0.9,
                opacity = 0.8,
                reversescale = False,
                autocolorscale = False,
                symbol = 'circle',
                line = dict(
                    width=1,
                    color='rgba(102, 102, 102)'),
            colorscale = 'Blues',
            cmin = merged_df_2018_2019['Surplus/Deficit'].min(),
            color = merged_df_2018_2019['Surplus/Deficit'],
            cmax = merged_df_2018_2019['Surplus/Deficit'].max(),
            colorbar_title="Surplus for the Club"
        )))

for i in range(len(final_DataFrame_2018_2019)):
    fig.add_trace(
        go.Scattergeo(
            lon = [final_DataFrame_2018_2019['Longitude_Left'][i], final_DataFrame_2018_2019['Longitude_Joined'][i]],
            lat = [final_DataFrame_2018_2019['Latitude_Left'][i], final_DataFrame_2018_2019['Latitude_Joined'][i]],
            mode = 'lines',
            hovertext = "Name: " + str(final_DataFrame_2018_2019['Name'][i]),
            line = dict(
                width = 1,
                color='darkgray')
        )
    )
    
fig.update_layout(
    title='Transfer Size: 20M to 50M (2018-2019)',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    mapbox_style="open-street-map"
    )

plot(fig, filename = 'Transfer_Size_20M_50M_2018_2019.html')

'Transfer_Size_20M_50M_2018_2019.html'